In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

# path to json file that stores MFCCs and genre labels for each processed segment
DATA_PATH = "mfcc_data_20s.json"

def load_data(data_path):
    """Loads training dataset from json file.

        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    X = np.array(data["mfccs"])
    y = np.array(data["labels"])
    genres = data["genres"]

    print("Data succesfully loaded!")

    return  X, y, genres

In [2]:
# load data
X, y, genres = load_data(DATA_PATH)

Data succesfully loaded!


In [3]:
# create train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [4]:
# build network topology
model = keras.Sequential([

    # input layer
    keras.layers.Flatten(input_shape=(X.shape[1], X.shape[2])),

    # 1st dense layer
    keras.layers.Dense(512, activation='relu'),

    # 2nd dense layer
    keras.layers.Dense(256, activation='relu'),

    # 3rd dense layer
    keras.layers.Dense(64, activation='relu'),

    # output layer
    keras.layers.Dense(len(genres), activation='softmax')
])

# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 17240)             0         
                                                                 
 dense (Dense)               (None, 512)               8827392   
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 64)                16448     
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 8,975,818
Trainable params: 8,975,818
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=50)

Epoch 1/50
22/22 [==============================] - 3s 95ms/step - loss: 34.6969 - accuracy: 0.2275 - val_loss: 15.9691 - val_accuracy: 0.2600
Epoch 2/50
22/22 [==============================] - 2s 86ms/step - loss: 9.6708 - accuracy: 0.4192 - val_loss: 9.3555 - val_accuracy: 0.3633
Epoch 3/50
22/22 [==============================] - 2s 86ms/step - loss: 4.3829 - accuracy: 0.5737 - val_loss: 7.4022 - val_accuracy: 0.3900
Epoch 4/50
22/22 [==============================] - 2s 86ms/step - loss: 2.3482 - accuracy: 0.7353 - val_loss: 9.1727 - val_accuracy: 0.4233
Epoch 5/50
22/22 [==============================] - 2s 86ms/step - loss: 1.6166 - accuracy: 0.7811 - val_loss: 7.8180 - val_accuracy: 0.3933
Epoch 6/50
22/22 [==============================] - 2s 90ms/step - loss: 0.8820 - accuracy: 0.8383 - val_loss: 8.8597 - val_accuracy: 0.3700
Epoch 7/50
22/22 [==============================] - 2s 92ms/step - loss: 0.6479 - accuracy: 0.8770 - val_loss: 8.1465 - val_accuracy: 0.4200
Epoch 8/50


In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=64)
print("test loss, test acc:", results)

In [ ]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for tests")
predictions = model.predict(X_test)

In [ ]:
correct = [0 for genre in genres]
pred_data = [[0 for g in genres] for genre in genres]
genre_count = [0 for genre in genres]
for i in range(predictions.shape[0]):
    max_confidence = max(predictions[i])
    pred = np.argmax(predictions[i])
    actual = y_test[i]
    
    genre_count[actual] += 1
    pred_data[actual][pred] += 1
    
    if pred == actual:
        correct[actual] += 1
    print('Prediction:', genres[pred], '\tActual:', genres[actual], '\nConfidence:', max(predictions[i]),'\n')

In [ ]:
curr_genre = 0
for c,g in zip(correct,genre_count):
    print(genres[curr_genre], ': ', c / g * 100,'% correct')
    print('Prediction distribution: ', pred_data[curr_genre],'\n')
    curr_genre += 1

print('\nTotal: ', sum(correct) / sum(genre_count) * 100, '% correct')